In [90]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from itertools import permutations

options = np.array([4,2, 18])
costs = np.array([118, 60, 756])
strategies = np.array(list(permutations(np.arange(len(options)))))
avg = (15+6)/6.0

def run_strategy(options, costs, strategy, avg=3.5, starting_cash = 0.0, verbose=False):
    s = starting_cash
    total_steps = 0
    for i in strategy:
        # how many times would we have to throw the dice to be able to buy the next option?
        next_cost = costs[i] - s
        t = np.ceil(next_cost/avg)
        total_steps += t
        if verbose:
            print(f"{total_steps}: Ran {t} steps with avg {avg:.2f}")
        # how much cash would be left afterwards?
        s = t*avg - next_cost
        if verbose:
            print(f"{total_steps} We have {s} left after buying")
        # update avg
        avg += options[i]/6.0
        if verbose:
            print(f"{total_steps} avg now at {avg:.2f}")
    return total_steps, s

def batch_eval(options, costs, strategies, avg=3.5):
    # run all strategies and record the results
    r = np.zeros((len(strategies),3),"float32")
    for i, strat in enumerate(strategies):
        r[i,:2] = run_strategy(options, costs, strat, avg)

    # compute how much money you would have if all
    # strategies were run for the longest time of
    # all:
    final_avg = avg + options.sum()/6.0
    r[:,2] = (r[:,0].max()-r[:,0])*final_avg + r[:,1]
    data = pd.DataFrame(r,columns=["Runs","e","Final cash"])
    data = data.drop("e", axis=1)
    data["strategy"] = [[options[x] for x in s] for s in strategies]
    data = data.sort_values("Final cash", ascending=False)
    return data

r = batch_eval(options, costs, strategies)
r

,Runs,Final cash,strategy
2,216.0,202.500000,"[2, 4, 18]"
0,217.0,198.500000,"[4, 2, 18]"
1,224.0,143.166672,"[4, 18, 2]"
3,232.0,82.833336,"[2, 18, 4]"
5,243.0,3.166667,"[18, 2, 4]"
4,243.0,2.833333,"[18, 4, 2]"


In [49]:
run_strategy(options, costs, [3,0,1,2], verbose=True)

44.0: Ran 44.0 steps with avg 3.50
44.0 We have 1.0 left after buying
44.0 avg now at 4.33
53.0: Ran 9.0 steps with avg 4.33
53.0 We have 3.0 left after buying
53.0 avg now at 4.50
61.0: Ran 8.0 steps with avg 4.50
61.0 We have 2.0 left after buying
61.0 avg now at 4.67
69.0: Ran 8.0 steps with avg 4.67
69.0 We have 0.3333333333333357 left after buying
69.0 avg now at 4.83


(69.0, 0.3333333333333357)

In [50]:
run_strategy(options, costs, [3,2,0,1], verbose=True)

44.0: Ran 44.0 steps with avg 3.50
44.0 We have 1.0 left after buying
44.0 avg now at 4.33
53.0: Ran 9.0 steps with avg 4.33
53.0 We have 1.0 left after buying
53.0 avg now at 4.50
61.0: Ran 8.0 steps with avg 4.50
61.0 We have 0.0 left after buying
61.0 avg now at 4.67
69.0: Ran 8.0 steps with avg 4.67
69.0 We have 0.3333333333333357 left after buying
69.0 avg now at 4.83


(69.0, 0.3333333333333357)